In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [2]:
import pandas as pd
import numpy as np

import src.preprocessing as pre
import plotly.graph_objects as go
import src.utils as utils
from torch.utils.data import DataLoader
import src.table as table
import src.graphs as graphs

In [3]:
train, test, mappings, meta = pre.separate_year('../data/meta.json','../data/y.csv','../data/mappings.json')

In [4]:
model = utils.create_model(train,num_epochs=1)

Epoch [1/200], Loss: 4393775.4190


In [5]:
preds = utils.predict_gridded_harvest(model,test)

In [6]:
actuals = test.Y.detach().numpy()

In [7]:
production_plan = pre.decode(meta,mappings).reset_index(drop=True)

In [8]:
idx_dict = table.create_indices_dict(production_plan)

In [9]:
transplant_weeks = production_plan.WeekTransplanted.to_numpy()
year_preds, year_actuals = table.season_shift(transplant_weeks,preds,actuals)
year_preds.shape,year_actuals.shape

((139, 20, 51), (139, 51))

In [10]:
fig1 = graphs.graph_harvest(preds,actuals)
fig2 = graphs.graph_harvest(year_preds,year_actuals)

In [11]:
fig3 = graphs.graph_harvest_cumsum(preds,actuals)

In [12]:
types_i_want_lol = ['Pera Roja','Uva Amarilla']

In [ ]:
preds_sum,actuals_sum = table.filter_preds(preds,actuals,'Type',types_i_want_lol,idx_dict)

In [14]:
fig4 = graphs.graph_harvest_stacked(preds_sum,actuals_sum,types_i_want_lol)

In [15]:
fig1.show()

In [16]:
fig2.show()

In [17]:
fig3.show()

In [18]:
fig4.show()
